In [1]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

#lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['BELUSDT','RENUSDT','KNCUSDT','CVCUSDT','GTCUSDT','LQTYUSDT','STGUSDT','BAKEUSDT','AUDIOUSDT','MAVUSDT','BNTUSDT','LPTUSDT','LEVERUSDT','XRPUSDT']
lista=['YGGUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data = md.estrategia_triangulos(symbol, tp_flag = False, print_lines_flag = imprimo)
        #data,porcentajeentrada = md.estrategia_adrian(symbol,tp_flag = False)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_atrapes(symbol,tp_flag = True, debug=debug)
        #######################################################################
        #resultado = md.backtesting(data, plot_flag = imprimo)
        resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatte

YGGUSDT - Return [%]: 0.39% - # Trades: 3 - Profit Factor: nan - Win Rate [%]: 100.0


In [1]:
import requests
import pandas as pd
import json
API_KEY = '4c9c0645-49c7-48c3-9a42-e7a2f94d448f'
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
HEADERS = {'Accepts': 'application/json',
           'X-CMC_PRO_API_KEY': API_KEY}
PARAMS = {'convert': 'USD',
          'limit': 1500}
resp = requests.get(url, headers=HEADERS, params=PARAMS)
def get_data(resp):
    data = json.loads(resp.content)['data']
    rows = list()
    for item in data:
        rows.append([
                item['cmc_rank'],
                item['id'],
                item['name'],
                item['symbol'],
                item['slug'], 
                item['quote']['USD']['market_cap'],
                item['quote']['USD']['volume_24h'],
                item['date_added']])
    df = pd.DataFrame(rows, columns=['cmc_rank','id','name','symbol','slug','market_cap','volume_24h','date_added'])
    df.index.name = 'id'
    return(df)
if resp.status_code == 200:
    df = get_data(resp)
    if not df.empty:
        df.to_csv('crypto_latests.csv')
        print("Archivo con datos de CoinMarketCap gruadado crypto_latests.csv")
        df2 = pd.read_csv('crypto_latests.csv', index_col=0)
    else:
        print("df de CoinMarketCap vacío. No se actualizó crypto_latests.csv")
else:
    print(resp.status_code)

Archivo con datos de CoinMarketCap gruadado crypto_latests.csv


In [ ]:
import modulos as md
from numerize import numerize
import pandas as pd

df2 = pd.read_csv('crypto_latests.csv', index_col=0)
symbol='CYBERUSDT'
for index, row in df2.iterrows():
    if row['symbol'] == (symbol[0:symbol.find('USDT')]).upper():       
        print(f"Symbol: {symbol} - Ranking: {row.cmc_rank} - Market cap: {numerize.numerize(row.market_cap)} - Vol24h: {numerize.numerize(row.volume_24h)} - date_added: {row.date_added}")

vo=md.volumeOf24h(symbol)
print(f"Vol24h: {numerize.numerize(vo)}")   
   